In [1]:
# Importando o Pandas

import pandas as pd

In [2]:
# Lendo o arquivo csv

df = pd.read_csv('train (1).csv')

In [3]:
# Excluindo features não importantes (Name e PassengerId) e com muitos valores NULL (Cabin)

df = df.drop(columns=['Name', 'Cabin', 'PassengerId'])

In [4]:
# Calculando a mediana da idade

mediana_age = df['Age'].median()
print(mediana_age)

28.0


In [5]:
# Substituindo valores NA de age por sua mediana

df['Age'] = df['Age'].fillna(mediana_age)

In [6]:
# Calculando a moda de Embarked

moda_embarcked = df['Embarked'].mode()[0]
moda_embarcked

'S'

In [7]:
# Substituindo valores NA de Embarked por sua mediana

df['Embarked'] = df['Embarked'].fillna(moda_embarcked)

In [8]:
# Verificando que existem muitos valores distintos em Ticket

print("Número de valores únicos em 'Ticket':", df['Ticket'].nunique())

Número de valores únicos em 'Ticket': 681


In [9]:
#Transformar a coluna categórica Ticket em uma variável numérica

freq = df['Ticket'].value_counts(normalize=True).to_dict()
df['Ticket_FreqEncoded'] = df['Ticket'].map(freq)

In [10]:
# Removendo a coluna ticket

df = df.drop('Ticket', axis=1)

In [11]:
# Aplicando o encoder em Embarked

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first', sparse_output=False)

encoded_col = encoder.fit_transform(df[['Embarked']])

encoded_df = pd.DataFrame(encoded_col, columns = encoder.get_feature_names_out(['Embarked']))

df = pd.concat([df, encoded_df], axis=1)
df = df.drop('Embarked', axis=1)


In [12]:
# Aplicando enoder em Sex

encoded_col = encoder.fit_transform(df[['Sex']])

encoded_df = pd.DataFrame(encoded_col, columns = encoder.get_feature_names_out(['Sex']))

df = pd.concat([df, encoded_df], axis=1)
df = df.drop('Sex', axis=1)

In [13]:
# Separando em X e y

X = df.drop('Survived', axis=1)
y = df['Survived']

In [14]:
# Treinando os modelos de classificação

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

modelo_lr = LogisticRegression()

modelo_lr.fit(X_scaled, y)

modelo_dtc = DecisionTreeClassifier()

modelo_dtc.fit(X_scaled, y)

modelo_knn = KNeighborsClassifier()

modelo_knn.fit(X_scaled, y)

modelo_rfc = RandomForestClassifier()

modelo_rfc.fit(X_scaled, y)

RandomForestClassifier()

In [15]:
# Lendo os arquivos de teste

df_teste = pd.read_csv('test.csv')
df_teste_resultado = pd.read_csv('gender_submission.csv')

In [16]:
# Removendo colunas desnecessárias ou maléficas aos modelos

df_teste = df_teste.drop(columns=['Name', 'Cabin'])

In [17]:
# Verificando valores nulos

df_teste.isna().sum()

PassengerId     0
Pclass          0
Sex             0
Age            86
SibSp           0
Parch           0
Ticket          0
Fare            1
Embarked        0
dtype: int64

In [18]:
# Calculando a mediana da idade e substituindo valores nulos

mediana_age = df_teste['Age'].median()
print(mediana_age)

df_teste['Age'] = df_teste['Age'].fillna(mediana_age)

27.0


In [19]:
# Substituindo valores de Ticket categóricos para numéricos por meio da frequência

freq = df_teste['Ticket'].value_counts(normalize=True).to_dict()
df_teste['Ticket_FreqEncoded'] = df_teste['Ticket'].map(freq)

In [20]:
# Removendo Ticket

df_teste = df_teste.drop('Ticket', axis=1)

In [21]:
# Aplicando encoder ao Embarked

encoded_col = encoder.fit_transform(df_teste[['Embarked']])

encoded_df = pd.DataFrame(encoded_col, columns = encoder.get_feature_names_out(['Embarked']))

df_teste = pd.concat([df_teste, encoded_df], axis=1)
df_teste = df_teste.drop('Embarked', axis=1)

In [22]:
# Aplicando encoder a Sex

encoded_col = encoder.fit_transform(df_teste[['Sex']])

encoded_df = pd.DataFrame(encoded_col, columns = encoder.get_feature_names_out(['Sex']))

df_teste = pd.concat([df_teste, encoded_df], axis=1)
df_teste = df_teste.drop('Sex', axis=1)

In [23]:
# Fazendo merge entre os dataframes de teste

df_final = pd.merge(df_teste, df_teste_resultado[['PassengerId', 'Survived']], on='PassengerId', how='left')

In [24]:
# Corrigindo valores nulos de Fare pela mediana

df_final['Fare'] = df_final['Fare'].fillna(df['Fare'].median())

print(df_final.isnull().sum())

PassengerId           0
Pclass                0
Age                   0
SibSp                 0
Parch                 0
Fare                  0
Ticket_FreqEncoded    0
Embarked_Q            0
Embarked_S            0
Sex_male              0
Survived              0
dtype: int64


In [25]:
# Dividindo em X teste e y teste

X_test = df_final.drop(['Survived', 'PassengerId'], axis=1)
y_test = df_final['Survived']

In [26]:
# Normalizando

X_test_scaled = scaler.transform(X_test)

In [27]:
# Realizando a predição

pred_lr = modelo_lr.predict(X_test_scaled)

In [28]:
# Realizando a predição

pred_dtc = modelo_dtc.predict(X_test_scaled)

In [29]:
# Realizando a predição

pred_knn = modelo_knn.predict(X_test_scaled)

In [30]:
# Realizando a predição

pred_rfc = modelo_rfc.predict(X_test_scaled)

In [31]:
# Calculando a acurácia

from sklearn.metrics import accuracy_score

acc_lr = accuracy_score(y_test, pred_lr)

acc_dtc = accuracy_score(y_test, pred_dtc)

acc_knn = accuracy_score(y_test, pred_knn)

acc_rfc = accuracy_score(y_test, pred_rfc)

In [32]:
# Imprimindo as acurácias

print(f'Accurancy do Linear regression: {acc_lr}\nAccurancy do DTC: {acc_dtc}\nAccurancy KNN: {acc_knn}\nAccurzncy RFC: {acc_rfc}')

Accurancy do Linear regression: 0.9354066985645934
Accurancy do DTC: 0.7751196172248804
Accurancy KNN: 0.854066985645933
Accurzncy RFC: 0.861244019138756


In [33]:
# Imprimindo a matriz de confusão da regressão logística

from sklearn.metrics import confusion_matrix

cm_lr = confusion_matrix(y_test, pred_lr)
print("Matriz de confusão - Regressão Logística:")
print(cm_lr)

Matriz de confusão - Regressão Logística:
[[248  18]
 [  9 143]]


Nosso melhor modelo foi a Regressão Logística com 93% de precisao